In [ ]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

# Set open-meteo client
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Set up the API and parameters
url = "https://archive-api.open-meteo.com/v1/era5"
params = {
    # define the Columbia University's latitude and longitude 
    "latitude": 40.807384,
    "longitude": -73.963036,
    "start_date": "2024-01-01",
    "end_date": "2025-10-31",
    "hourly": [
        "temperature_2m",
        "wind_speed_10m",
        "precipitation",
        "snowfall",
        "snow_depth"
    ],
    "timezone": "America/New_York",
    "utm_source": "chatgpt.com",
}

# requests content from API
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()} s")

hourly = response.Hourly()

hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation  = hourly.Variables(2).ValuesAsNumpy()
hourly_snowfall       = hourly.Variables(3).ValuesAsNumpy()
hourly_snow_depth     = hourly.Variables(4).ValuesAsNumpy()

# create timeseries index
hourly_data = {
    "time": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left",
    )
}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["precipitation"]  = hourly_precipitation
hourly_data["snowfall"]       = hourly_snowfall
hourly_data["snow_depth"]     = hourly_snow_depth

# Change to Dataframe
hourly_dataframe = pd.DataFrame(data=hourly_data)

print("\nHourly data (head):")
print(hourly_dataframe.head())
print("\nRows:", len(hourly_dataframe))

# Save CSV data in the data folder
out_path = "../data/openmeteo_cu_hourly_2024_01_to_2025_10_from_client.csv"
hourly_dataframe.to_csv(out_path, index=False)
print(f"\nSaved to: {out_path}")


Coordinates: 40.808433532714844°N -74.0198974609375°E
Elevation: 48.0 m asl
Timezone difference to GMT+0: -18000 s

Hourly data (head):
                       time  temperature_2m  wind_speed_10m  precipitation  \
0 2024-01-01 05:00:00+00:00           2.331        9.021574            0.0   
1 2024-01-01 06:00:00+00:00           2.931       10.883676            0.0   
2 2024-01-01 07:00:00+00:00           2.981       10.086427            0.0   
3 2024-01-01 08:00:00+00:00           2.731        8.089993            0.0   
4 2024-01-01 09:00:00+00:00           0.531        5.588703            0.0   

   snowfall  snow_depth  
0       0.0         0.0  
1       0.0         0.0  
2       0.0         0.0  
3       0.0         0.0  
4       0.0         0.0  

Rows: 16080

Saved to: ../data/openmeteo_cu_hourly_2024_01_to_2025_10_from_client.csv
